# import dataset

In [1]:
from datasets import load_dataset
dataset = load_dataset("SLPRL-HUJI/HebDB", "YV_pre", split='train', streaming=True, cache_dir='datasets')

Resolving data files:   0%|          | 0/608 [00:00<?, ?it/s]

In [2]:
dataset.features

{'fname': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None),
 'n_samples': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'normalized_text': Value(dtype='string', id=None),
 'score': Value(dtype='float64', id=None),
 'raw': {'end_sec': Value(dtype='float64', id=None),
  'fname': Value(dtype='string', id=None),
  'start_sec': Value(dtype='float64', id=None)},
 'is_raw': Value(dtype='bool', id=None),
 'source': Value(dtype='string', id=None)}

In [3]:
print(dataset.take(1))

IterableDataset({
    features: ['fname', 'audio', 'n_samples', 'text', 'normalized_text', 'score', 'raw', 'is_raw', 'source'],
    n_shards: 15
})


---
# Preprocessing

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [21]:
def tokenization(examples):
    print('tokenization')
    examples["normalized_text2"] = tokenizer(examples["normalized_text"], padding="max_length")

In [20]:
dataset_pre = dataset.map(tokenization, batched=True)

In [7]:
dataset_pre = dataset_pre.with_format("torch")

In [8]:
dataset_pre.features

---
# Dataloader

In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

In [22]:
dataloader = DataLoader(dataset, collate_fn=DataCollatorForLanguageModeling(tokenizer))

KeyboardInterrupt: 

In [13]:
for batch in dataloader:
    print(batch)
    break

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['fname', 'audio', 'n_samples', 'text', 'normalized_text', 'score', 'raw', 'is_raw', 'source']